In [2]:
! pip install -q lmdb pillow torchvision nltk natsort

In [3]:
! git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 499 (delta 208), reused 200 (delta 200), pack-reused 274 (from 1)
Receiving objects: 100% (499/499), 3.05 MiB | 38.14 MiB/s, done.
Resolving deltas: 100% (308/308), done.


In [4]:
%cd deep-text-recognition-benchmark/

/content/deep-text-recognition-benchmark


In [5]:
! gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM

Downloading...
From (original): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
From (redirected): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM&confirm=t&uuid=0cae3436-337a-4ce5-932d-cff3c29a9845
To: /content/deep-text-recognition-benchmark/plate_img-train.zip
100% 196M/196M [00:04<00:00, 41.4MB/s]


In [6]:
! gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I

Downloading...
From (original): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
From (redirected): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I&confirm=t&uuid=1a1acf26-98de-4f5f-b938-2c0e4aef5a93
To: /content/deep-text-recognition-benchmark/plate_img-validation.zip
100% 27.2M/27.2M [00:01<00:00, 22.0MB/s]


In [ ]:
!unzip /content/deep-text-recognition-benchmark/plate_img-train.zip -d /content/Dataset/train/
!unzip /content/deep-text-recognition-benchmark/plate_img-validation.zip -d /content/Dataset/validation/

In [9]:
import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/train/train'
output_file = '/content/Data/gt.txt'
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(train_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            chars.append((xmin, name))

        # Sort characters from left to right
        chars = sorted(chars, key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])

        lines.append(f'{image_path}\t{label}')

# Save to gt.txt
with open(os.path.join(train_dir, output_file), 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(line + '\n')

print(f'Done! {len(lines)} items written to {output_file}')


Done! 19381 items written to /content/Data/gt.txt


In [10]:
import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/validation/validation'
output_file = '/content/Data/gt_validation.txt'
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(train_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            chars.append((xmin, name))

        # Sort characters from left to right
        chars = sorted(chars, key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])

        lines.append(f'{image_path}\t{label}')

# Save to gt.txt
with open(os.path.join(train_dir, output_file), 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(line + '\n')

print(f'Done! {len(lines)} items written to {output_file}')


Done! 2805 items written to /content/Data/gt_validation.txt


In [12]:
import os
import shutil

source_dir = '/content/Dataset/train/train'
destination_dir = '/content/Data/train'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    if filename.lower().endswith('.jpg'):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        shutil.move(src_path, dst_path)

print("Done train data!")

source_dir = '/content/Dataset/validation/validation'
destination_dir = '/content/Data/validation'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    if filename.lower().endswith('.jpg'):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        shutil.move(src_path, dst_path)

print("Done validation data !")


Done train data!
Done validation data !


In [26]:
!python3 train.py \
--train_data data_lmdb_release/training --valid_data data_lmdb_release/validation \
--select_data / --batch_ratio 1 \
--Transformation None --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction CTC


Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
dataset_root:    data_lmdb_release/validation	 dataset: /
Traceback (most recent call last):
  File "/content/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/content/deep-text-recognition-benchmark/train.py", line 35, in train
    valid_dataset, valid_dataset_log = hierarchical_dataset(root=opt.valid_data, opt=opt)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/deep-text-recognition-benchmark/dataset.py", line 130, in hierarchical_dataset
    concatenated_dataset = ConcatDataset(dataset_list)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 328, in __init__
    assert len(self.datasets) > 0, "datasets should not be an empty iterable"  # type: ignore[arg-type]